## Query header

In this notebook, we show several ways to query the fits headers with Gen3\
Craig Lage - 03-Jun-21

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
dayObs = 20230126
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.target_name)


In [ ]:
print("ExpId,            RA                   Dec")
for exp in range(2022101800025, 2022101800047):
    mData = butler.get('raw.metadata', detector=0, exposure=exp)
    print(exp, mData['RASTART'], mData['DECSTART'])
    

In [ ]:
expId = 2022110800097
mBias = butler.get('raw', detector=0, exposure=expId)

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = mBias.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=20000, vmax=30000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


In [ ]:
ycoord = 3500

plt.figure(figsize=(4,8))
plt.subplots_adjust(hspace=0.5)
plotCounter = 1
plt.suptitle("AuxTel darks with shorter exposure time.")
for expId in [2022110900030, 2022110900095, 2022110900150]:
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    rawData = butler.get('raw', detector=0, exposure=expId)
    arr = rawData.image.array
    plt.subplot(3,1,plotCounter)
    plt.plot(arr[ycoord,:])
    plt.title(f"{expId}, Dark Exposure Time = {mData['EXPTIME']}, Ycoord = {ycoord}")
    plotCounter += 1

In [ ]:
exposureList[:][27]

In [ ]:
exposureList[0][1].fields

In [ ]:
from lsst.daf.butler import Butler


nTot=0
for [expId,record] in exposureList:
    nExp = 0
    for detector in range(9):
        try:
            mData = butler.get('raw.metadata', detector=detector, exposure=expId)
        except:
            nTot += 1
            nExp += 1
            #print(f"Detector {detector} from expId {expId} missing")
    if nExp != 0:
        print(f" expId {expId} is missing {nExp} files.")
print(f" Total of {n} exposures missing")

In [ ]:
expId = 2023011800017
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    #if 'E' in key:
    print(key, mData[key])

In [ ]:
expId = 2022040600654
exp = butler.get('quickLookExp', detector=0, exposure=expId)


In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
mBias = butler.get('bias', detector=0, exposure=2022092900947)